In [2]:
import dropbox
import shutil
import getpass
import os
from enum import Enum

class Model(Enum):
    OPEN_AI = 1
    LLAMA = 2
    NOMIC = 3

QUERIES = [
    "How to deal with a memory leak in prod?",
    "What are the responsibilities of the First Captain during a fire?",
    "What are the First Captain's responsibilities during a fire?",
    "What should I do if there are problems with the haskell quiz engine?",
    "What are my responsibilites when on call?",
    "What do I do if my time off conflicts with being on call?",
]

DOWNLOAD_FOLDER = ".content"
TEST_CONTENT_DROPBOX_FOLDER = "LLM Doc Exp Test Content"
EMBEDDINGS_MODEL = Model.NOMIC
LLM_MODEL = Model.LLAMA
RESET_DROPBOX = False
RESET_VECTOR_STORE = False
TEST_QUERY = QUERIES[0]

In [3]:
# import requests
# import sseclient
# import json


# def get_stream(app, query):
#     full_response = ""
#     url = "http://127.0.0.1:8000/query/stream_events/"
#     response = requests.post(url, json={"input": {"query": query}}, stream=True)
#     client = sseclient.SSEClient(response)
#     for event in client.events():
#         output = json.loads(event.data)
#         if output["event"] == "on_llm_stream":
#             if chunk := output["data"].get("chunk"):
#                 full_response += chunk
#                 yield full_response
#         elif output["event"] == "on_chain_end":
#             if source_documents := output["data"]["output"].get("source_documents"):
#                 yield output["data"]["output"]["result"]

# for chunk in get_stream(None, TEST_QUERY):
#     print(chunk, flush=True)


# from langserve import RemoteRunnable
# query_chain = RemoteRunnable("http://127.0.0.1:8000/query/")

# async for msg in query_chain.astream({"query": QUERIES[0]}):
#     print(msg, end="", flush=True)


In [4]:
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = {
    Model.OPEN_AI: OpenAIEmbeddings,
    Model.LLAMA: lambda: OllamaEmbeddings(model="llama3"),
    Model.NOMIC: lambda: OllamaEmbeddings(model="nomic-embed-text"),
}

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import Ollama

llms = {
    Model.OPEN_AI: lambda: ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    Model.LLAMA: lambda: Ollama(model="llama3", temperature=0),
    Model.NOMIC: lambda: Ollama(model="llama3", temperature=0),
}


In [6]:
if RESET_DROPBOX:
    dbx = dropbox.Dropbox(getpass.getpass("Dropbox API Key:"))

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Open AI API Key:")

In [8]:
# Ensure local download folder exists, and delete its contents

def create_download_folder():
    if not os.path.exists(DOWNLOAD_FOLDER):
        os.makedirs(DOWNLOAD_FOLDER)

def clear_downloads_folder():
    for filename in os.listdir(DOWNLOAD_FOLDER):
        file_path = os.path.join(DOWNLOAD_FOLDER, filename)
        print(file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


create_download_folder()
if RESET_DROPBOX:
    clear_downloads_folder()

# Filter dropbox paper docs and download to content file

Download selected test paper docs using the [Dropbox API](https://www.dropbox.com/developers/documentation/http/documentation#paper-docs-download) and [Python SDK](https://dropbox-sdk-python.readthedocs.io/en/latest/index.html)

The dashboard for the Dropbox App used to do this can be found [here](https://www.dropbox.com/developers/apps/info/la3hq2wkhl5wx4m)

In [9]:
# from dropbox.paper import ExportFormat
# folders_to_match = set(["Engineering", "Fires"])

# print("Getting doc ids")
# doc_ids = dbx.paper_docs_list().doc_ids

# print("Getting docs in matching folders")
# docs_ids_in_folder = [doc_id for doc_id in doc_ids if folders_to_match.issubset(set([folder.name for folder in dbx.paper_docs_get_folder_info(doc_id).folders or []]))]

# print("Getting doc titles")
# doc_titles = {doc_id: dbx.paper_docs_download(doc_id, ExportFormat('markdown'))[0].title for doc_id in docs_ids_in_folder}


In [10]:
from dropbox.paper import ExportFormat, ListPaperDocsFilterBy

def get_file_path(doc_id):
    return os.path.join(DOWNLOAD_FOLDER, f"{doc_titles[doc_id]}.md")

def download_doc(doc_id):
    result = dbx.paper_docs_download_to_file(get_file_path(doc_id), doc_id, ExportFormat('markdown'))
    print(f"- downloaded '{result.title}'")
    return result

if RESET_DROPBOX:
    print("Retrieving document IDs")
    doc_ids = dbx.paper_docs_list(filter_by=ListPaperDocsFilterBy.docs_created).doc_ids
    print(f"- {len(doc_ids)} documents found")

    print("Filtering documents in folder")
    docs_ids_in_folder = [doc_id for doc_id in doc_ids if TEST_CONTENT_DROPBOX_FOLDER in [folder.name for folder in dbx.paper_docs_get_folder_info(doc_id).folders or []]]
    print(f"- {len(docs_ids_in_folder)} documents found in folder")

    print("Retrieving document titles")
    doc_titles = {doc_id: dbx.paper_docs_download(doc_id, ExportFormat('markdown'))[0].title for doc_id in docs_ids_in_folder}

    print("Downloading documents")
    results = [download_doc(doc_id) for doc_id in docs_ids_in_folder]
    print("Download complete")

# Simple RAG Q&A using the downloaded files

Uses [this repo](https://github.com/AI-Maker-Space/LLM-Ops-Cohort-1/blob/main/Week%201/Tuesday/Barbie_Retrieval_Augmented_Question_Answering_(RAQA)_Assignment%20(Assignment%20Version).ipynb) as a reference.


Additional Resources:
https://github.com/zylon-ai/private-gpt/issues/358#issuecomment-1563663500
https://python.langchain.com/docs/integrations/vectorstores/starrocks/
https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/

In [11]:
from langchain_community.document_loaders.markdown import UnstructuredMarkdownLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownTextSplitter
from langchain_text_splitters import Language


In [100]:
loader = DirectoryLoader(DOWNLOAD_FOLDER, glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)

raw_documents = loader.load()


In [13]:

# headers_to_split_on = [
#     ("#", "Header 1"),
#     ("##", "Header 2"),
# ]

# # MD splits
# markdown_splitter = MarkdownHeaderTextSplitter(
#     headers_to_split_on=headers_to_split_on, strip_headers=False
# )

# loader.load_and_split(text_splitter=markdown_splitter)
# MarkdownTextSplitter()

# # Char-level splits
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk_size = 250
# chunk_overlap = 30
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size, chunk_overlap=chunk_overlap
# )

# # Split
# splits = text_splitter.split_documents(md_header_splits)
# splits

In [14]:

# text_splitter = MarkdownTextSplitter(
#     chunk_size = 1000, # the character length of the chunk
#     chunk_overlap = 100, # the character length of the overlap between chunks
#     length_function = len, # the length function
# )

text_splitter = RecursiveCharacterTextSplitter.from_language(Language.MARKDOWN, 
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function
)

documents = text_splitter.split_documents(raw_documents)

In [15]:
print(documents[0])
print(documents[1])

page_content='🔥 Fire triage drill\n\nNote: This document is not intended to be consumed by itself. I’m personally introducing fire fighters to this drill at the start of their on-call cycle, and the document is here for a) reference for me b) reference for fire fighters after our meeting.\n\nThe fire drill starts at the NewRelic APM main screen for the Monolith. Go through the numbers on the image, reading their description below, and follow the arrows ➡️ .\n\nNewRelic\n\nMonolith NewRelic main screen\n\n🔗 Link\n\nRequest breakdown\nDid something start taking up a larger share of request time?\nExternal services\n➡️ Go to the External Services tab in the right and figure out which service\n\n\nDB\n➡️ Go to the Databases tab and see whether some transactions started spiking\n➡️ Open RDS Performance Insights for higher resolution data\n\n\nQueueing or Ruby Slowness\n➡️ Open Load Balancers Dashboard\n➡️ Opsworks time-based instances and load-based instances\nDo we have instances booting u

# Index Creation

https://python.langchain.com/docs/modules/data_connection/vectorstores/

In [16]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore

In [17]:
store = LocalFileStore(".cache")

core_embeddings_model = embeddings[EMBEDDINGS_MODEL]()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=core_embeddings_model.model
)

vector_store = FAISS.from_documents(documents, core_embeddings_model, normalize_L2=True)
vector_store.save_local(".vector_store")

In [18]:
# Example query on the vector store
print(vector_store.distance_strategy)

query = TEST_QUERY
embedding_vector = core_embeddings_model.embed_query(query)
# docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)
docs = vector_store.similarity_search_with_score_by_vector(embedding_vector, score_threshold=10.0)
# docs = vector_store.similarity_search_with_score_by_vector(embedding_vector, k=4)

for page in docs:
  print(page[0].dict())
  # print(f">>>{page.page_content}<<<")


DistanceStrategy.EUCLIDEAN_DISTANCE
{'page_content': '# 2016–09–30 Other (~resolved🎉?) Fires\n\nMemory leak in prod\n\nUpdate / Tentatively Resolved\nAppears that the cause was not in the suspect list below (but in the deploy) https://github.com/NoRedInk/NoRedInk/pull/13550 which is now the primary suspect with a fix deployed in https://github.com/NoRedInk/NoRedInk/pull/13703 that thus far appears to have removed network correlated memory spikes.\n\nThings we can do\n\nDeploy/rollback to production-2016-09-29-0434\n\nRevert the suspect PRs\n\nSuspect PRs\nFrom @Joshua L (sorted by suspicion)', 'metadata': {'source': '.content/Engineering/Fires/Old_Fires/2016–09–30_Other_(~resolved_)_Fires.md'}, 'type': 'Document'}
{'page_content': 'Solutions/thoughts\n\nFree memory usage then run again\nseems to work sometimes, othertimes does not\n\nOccasionally, it will build successfully\nunable to reproduce consistently\n\nWiping elm-stuff prior to running webpack has no success\n\nTried to increas

# Retrieval Chain

In [19]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain

In [20]:
llm = llms[LLM_MODEL]()

In [24]:
def get_embeddings():
    store = LocalFileStore(".cache")

    return CacheBackedEmbeddings.from_bytes_store(
        core_embeddings_model, store, namespace=core_embeddings_model.model
    )
def load_vector_store():
    embeddings = get_embeddings()
    return FAISS.load_local(
        ".vector_store", embeddings, allow_dangerous_deserialization=True
    )
loaded_vector_store = load_vector_store()
# retriever = vector_store.as_retriever()
retriever = loaded_vector_store.as_retriever(
    # search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.9},
)

In [25]:
handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True,
)

In [26]:
import langchain.chains.qa_with_sources.retrieval
import langchain.chains.retrieval_qa.base
from langchain.chains.combine_documents import create_stuff_documents_chain

In [31]:
result = qa_with_sources_chain({"query" : QUERIES[1]})
# from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough

# def get_context_from_documents(documents):
#     document_separator = "\n\n"
#     return document_separator.join(f"Document {i}:\n{doc.page_content}" for i, doc in enumerate(documents))


# context_chain = RunnablePassthrough.assign(context=lambda x: get_context_from_documents(x["source_documents"]))
# chain = qa_with_sources_chain | context_chain
# result = chain.invoke(TEST_QUERY)
print(result)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What are the responsibilities of the First Captain during a fire?', 'result': "I don't know.", 'source_documents': []}


In [ ]:
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder, ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.messages import AIMessage, HumanMessage

RESPONSE_TEMPLATE = """\
You are an expert programmer and problem-solver, tasked with answering any question \
about Langchain.

Generate a comprehensive and informative answer of 80 words or less for the \
given question based solely on the provided search results (URL and content). You must \
only use information from the provided search results. Use an unbiased and \
journalistic tone. Combine search results together into a coherent answer. Do not \
repeat text. Cite search results using [${{number}}] notation. Only cite the most \
relevant results that answer the question accurately. Place these citations at the end \
of the sentence or paragraph that reference them - do not put them all at the end. If \
different results refer to different entities within the same name, write separate \
answers for each entity.

You should use bullet points in your answer for readability. Put citations where they apply
rather than putting them all at the end.

If there is nothing in the context relevant to the question at hand, just say "Hmm, \
I'm not sure." Don't try to make up an answer.

Anything between the following `context`  html blocks is retrieved from a knowledge \
bank, not part of the conversation with the user. 

<context>
    {context} 
<context/>

REMEMBER: If there is no relevant information within the context, just say "Hmm, I'm \
not sure." Don't try to make up an answer. Anything between the preceding 'context' \
html blocks is retrieved from a knowledge bank, not part of the conversation with the \
user.\
"""

REPHRASE_TEMPLATE = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:"""

def format_docs(docs) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(doc_string)
    return "\n".join(formatted_docs)

def serialize_history(request):
    chat_history = request["chat_history"] or []
    converted_chat_history = []
    for message in chat_history:
        if message.get("human") is not None:
            converted_chat_history.append(HumanMessage(content=message["human"]))
        if message.get("ai") is not None:
            converted_chat_history.append(AIMessage(content=message["ai"]))
    return converted_chat_history


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(REPHRASE_TEMPLATE)
condense_question_chain = (
    CONDENSE_QUESTION_PROMPT | llm | StrOutputParser()
).with_config(
    run_name="CondenseQuestion",
)
conversation_chain = condense_question_chain | retriever
retriever_chain = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        conversation_chain.with_config(run_name="RetrievalChainWithHistory"),
    ),
    (
        RunnableLambda(itemgetter("question")).with_config(
            run_name="Itemgetter:question"
        )
        | retriever
    ).with_config(run_name="RetrievalChainWithNoHistory"),
).with_config(run_name="RouteDependingOnChatHistory")


context = (
    RunnablePassthrough.assign(docs=retriever_chain)
    .assign(context=lambda x: format_docs(x["docs"]))
    .with_config(run_name="RetrieveDocs")
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", RESPONSE_TEMPLATE),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
default_response_synthesizer = prompt | llm

chain = RunnablePassthrough.assign(chat_history=serialize_history) | context | default_response_synthesizer

chain.run({"question": "How to deal with a memory leak in prod?", "chat_history": []})


In [55]:
print(documents[1].page_content)
print("=====================")
prompt = f"""Please help clean the following content in preparation for embedding as part of a RAG LLM application:
- remove emojis
- expand abbreviations and acronyms of common words
- remove bold and italic text styling
- section headers should be formatted using hierarchical hashtags instead of being underlined
- any other cleaning optimisations

Your answer MUST be in valid and well structured markdown and delimited by triple backticks (```).
Your answer MUST contain ALL factual information, and should serve as a faithful functional replacement of the original.

CONTENT:
```
Example topic
=============

irrelevant note

**first topic**
first info
some info
- list item 1
- list item 2

**second topic**
more info
```

CLEANED:
```
# Example topic

## first topic
first information
some information
- list item 1
- list item 2

## second topic
more information
```

CONTENT:
```
{documents[1].page_content}
```

CLEANED:
"""
# print(documents[0].page_content)
print(llm.invoke(prompt))

Throughput
Do we have a huge spike in throughput? 200k RPM and above might mean we’re under a DDoS attack
➡️ Check out what to do here: +Firefighting Resources: Fighting-a-DDoS-attack

Error rate
Are error spiking? Like 1% or more
➡️ Go to Errors tab
➡️ Check out Bugsnag (finer grained timeline)

External Services

Services by total response time
Any service started taking a longer time than usual?
A Haskell service?
➡️ Go to the specific service’s dashboard in NewRelic and do a similar analysis there (what made it slow? etc)
➡️ Also, open the Haskell Services in Kubernete dashboard in Datadog, because Kubernetes might be at fault
What services are in Kubernetes?
Drafts
Quiz Engine
Tutorials
Here is the cleaned content:
```
# Throughput
## Huge Spike in Throughput?
If we have a throughput of 200k RPM or above, it might indicate a DDoS attack. Check out firefighting resources for guidance.

# Error Rate
## High Error Rates?
If error rates are spiking at 1% or more, go to the Errors tab 

In [80]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
d = raw_documents[7].copy()
print(d.page_content)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mandlamoyo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mandlamoyo/nltk_data...


Fires checklist

[ ] Join https://appear.in/nri-fix-the-fire
[ ] Join #fix-the-fire
[ ] If there’s no captain, elect a captain (preferably someone who is on call)
[ ] Use either the captain’s or firefighter’s checklist below

Captain Checklists & Info

Using QA

During fires, there is generally no explicit QA schedule as it is impossible to know exactly when fixes will be available. QA can be requested ad hoc.

When recovering from a fire and the code has been hot deployed or the maintenance page has been used, it’s advisable to do full QA on the production site when we think everything is back to normal.

If the site is severely degraded, it may be OK to deploy a fix without QA. Risking regressions isn’t a serious concern if the site is mostly unusable. That being said: Code Review should still be observed & the staging environment should still be used to test changes against production data when necessary. @blunderdome can support in finding the type of data you want to test somethin

In [91]:
import string
# https://medium.com/intel-tech/four-data-cleaning-techniques-to-improve-large-language-model-llm-performance-77bee9003625

text = raw_documents[1].copy().page_content

# Tokenization

cleaned_tokens = re.sub(r"[_\-]", " ", text) 
tokens = word_tokenize(text)

# noise removal
cleaned_tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]
cleaned_tokens = [re.sub(r"[\d]", "", token) for token in cleaned_tokens]
cleaned_tokens = filter(lambda x: x not in string.punctuation + string.whitespace, cleaned_tokens)
# emoji_pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags=re.UNICODE)
# text = emoji_pattern.sub(r'', text)

# normalization
cleaned_tokens = [token.lower() for token in cleaned_tokens]

# stopword removal
stop_words = set(stopwords.words("english"))
cleaned_tokens = [token for token in cleaned_tokens if token not in stop_words]

# lemmatization
lemmatizer = WordNetLemmatizer()
cleaned_tokens = [lemmatizer.lemmatize(token) for token in cleaned_tokens]

print(cleaned_tokens)

['firefighting', 'resource', 'big', 'list', 'alert', 'type', 'response', 'playbook', 'slack', 'engopschanges', 'information', 'manually', 'performed', 'operation', 'may', 'caused', 'alert', 'askteamfoxen', 'slack', 'channel', 'infrastructure', 'tooling', 'team', 'shout', 'loudly', 'concern', 'infrastructure', 'cluster', 'db', 'networking', 'owned', 'noredink', 'nridoit', 'collab', 'channel', 'doit', 'international', 'partnered', 'provide', 'u', 'debugging', 'consulting', 'leave', 'message', 'probably', 'instructed', 'open', 'ticket', 'median', 'ticket', 'response', 'time', 'minute', 'content', 'creation', 'service', 'documented', 'userfacing', 'request', 'path', 'github', 'admin', 'right', 'noredink', 'computer', 'helpy', 'club', 'admin', 'right', 'main', 'noredink', 'repo', 'need', 'admin', 'right', 'part', 'firefighting', 'response', 'may', 'login', 'github', 'noredinkfirefighter', 'find', 'credential', 'password', 'table', 'content', 'frequent', 'firefighting', 'measure', 'productio

In [ ]:
import pprint
print(f"{result['query']}\n")
pprint.pp(result['result'])
print()

pprint.pp([document.metadata['source'] for document in result['source_documents']])
# pprint.pp(result['source_documents'][0].page_content)

In [121]:
import re
import string
# from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langchain_core.documents import Document

stop_words = {
    "himself",
    "ve",
    "their",
    "ain",
    "needn",
    "not",
    "re",
    "she's",
    "did",
    "yourself",
    "was",
    "an",
    "that'll",
    "wasn",
    "to",
    "hers",
    "y",
    "itself",
    "you're",
    "you",
    "how",
    "your",
    "m",
    "but",
    "ourselves",
    "him",
    "o",
    "a",
    "me",
    "as",
    "his",
    "our",
    "isn",
    "needn't",
    "weren",
    "what",
    "shan",
    "shan't",
    "had",
    "whom",
    "mightn",
    "who",
    "and",
    "of",
    "aren't",
    "i",
    "some",
    "isn't",
    "all",
    "yourselves",
    "those",
    "you'll",
    "you've",
    "these",
    "that",
    "such",
    "don",
    "they",
    "the",
    "it's",
    "is",
    "why",
    "just",
    "can",
    "theirs",
    "won",
    "do",
    "it",
    "he",
    "there",
    "hadn",
    "for",
    "or",
    "then",
    "no",
    "s",
    "ma",
    "hadn't",
    "so",
    "having",
    "she",
    "been",
    "we",
    "by",
    "my",
    "be",
    "yours",
    "ll",
    "nor",
    "them",
    "here",
    "than",
    "d",
    "being",
    "herself",
    "this",
    "very",
    "too",
    "both",
    "don't",
    "if",
    "you'd",
    "themselves",
    "am",
    "its",
    "her",
    "t",
    "are",
    "ours",
    "myself",
}


def pre_embedding_process(text: str) -> str:
    # Tokenization
    cleaned_tokens = re.sub(r"[_\-]", " ", text) 
    tokens = word_tokenize(text)

    # noise removal
    cleaned_tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]
    cleaned_tokens = [re.sub(r"[\d]", "", token) for token in cleaned_tokens]
    cleaned_tokens = filter(lambda x: x not in string.punctuation + string.whitespace, cleaned_tokens)

    # normalization
    cleaned_tokens = [token.lower() for token in cleaned_tokens]

    # stopword removal
    # stop_words = set(stopwords.words("english"))
    cleaned_tokens = [token for token in cleaned_tokens if token not in stop_words]

    # lemmatization
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in cleaned_tokens]

    return " ".join(cleaned_tokens)


def preprocess_document(document: Document) -> Document:
    document_copy = document.copy()
    document_copy.page_content = pre_embedding_process(document.page_content)
    return document_copy

test_query = "What are the responsibilities of the First Captain during a fire?"
test_text = "The first to acknowledge the incident is designated as the fire captain, who is responsible for managing the response until the incident is closed. Note that the role of captain can be handed off at any time, but that a captain should always be present while the incident is ongoing. The first and foremost objective of this system is to ensure the availability of NoRedInk, enabling its reliable use by teachers and students: Prioritize effectiveness over adherence to process (roles may get blurry in the heat of a fire), and try to maintain a balance between short-term effectiveness (quick response time) and long-term effectiveness (avoid burn out).\n\nResponsibilities\n\nOn-call people are responsible for off hours work (e.g. nights & weekends):\n\nbeing available in the #fires on slack & phone after their normal hours (including weekends) & within 2 hours of their computer"

import pprint
pprint.pp(pre_embedding_process(test_query))
pprint.pp(test_text)
pprint.pp(pre_embedding_process(test_text))

'responsibility first captain during fire'
('The first to acknowledge the incident is designated as the fire captain, who '
 'is responsible for managing the response until the incident is closed. Note '
 'that the role of captain can be handed off at any time, but that a captain '
 'should always be present while the incident is ongoing. The first and '
 'foremost objective of this system is to ensure the availability of NoRedInk, '
 'enabling its reliable use by teachers and students: Prioritize effectiveness '
 'over adherence to process (roles may get blurry in the heat of a fire), and '
 'try to maintain a balance between short-term effectiveness (quick response '
 'time) and long-term effectiveness (avoid burn out).\n'
 '\n'
 'Responsibilities\n'
 '\n'
 'On-call people are responsible for off hours work (e.g. nights & weekends):\n'
 '\n'
 'being available in the #fires on slack & phone after their normal hours '
 '(including weekends) & within 2 hours of their computer')
('first 

In [107]:
import uuid
from typing import Coroutine, List, Optional, Sequence, Any

from langchain_core.documents import Document
from langchain_text_splitters import TextSplitter

from langchain.retrievers import MultiVectorRetriever

from langchain_core.callbacks import AsyncCallbackManagerForRetrieverRun, CallbackManagerForRetrieverRun

class ParentDocumentPreprocessRetriever(MultiVectorRetriever):
    # Updates langchain.retrievers.ParentDocumentRetriever

    child_splitter: TextSplitter
    """The text splitter to use to create child documents."""

    """The key to use to track the parent id. This will be stored in the
    metadata of child documents."""
    parent_splitter: Optional[TextSplitter] = None
    """The text splitter to use to create parent documents.
    If none, then the parent documents will be the raw documents passed in."""

    child_metadata_fields: Optional[Sequence[str]] = None
    """Metadata fields to leave in child documents. If None, leave all parent document 
        metadata.
    """

    def add_documents(
        self,
        documents: List[Document],
        ids: Optional[List[str]] = None,
        add_to_docstore: bool = True,
    ) -> None:
        """Adds documents to the docstore and vectorstores.

        Args:
            documents: List of documents to add
            ids: Optional list of ids for documents. If provided should be the same
                length as the list of documents. Can be provided if parent documents
                are already in the document store and you don't want to re-add
                to the docstore. If not provided, random UUIDs will be used as
                ids.
            add_to_docstore: Boolean of whether to add documents to docstore.
                This can be false if and only if `ids` are provided. You may want
                to set this to False if the documents are already in the docstore
                and you don't want to re-add them.
        """
        if self.parent_splitter is not None:
            documents = self.parent_splitter.split_documents(documents)
        if ids is None:
            doc_ids = [str(uuid.uuid4()) for _ in documents]
            if not add_to_docstore:
                raise ValueError(
                    "If ids are not passed in, `add_to_docstore` MUST be True"
                )
        else:
            if len(documents) != len(ids):
                raise ValueError(
                    "Got uneven list of documents and ids. "
                    "If `ids` is provided, should be same length as `documents`."
                )
            doc_ids = ids

        docs = []
        full_docs = []
        for i, doc in enumerate(documents):
            _id = doc_ids[i]
            sub_docs = self.child_splitter.split_documents([doc])
            sub_docs = [preprocess_document(doc) for doc in sub_docs]
            if self.child_metadata_fields is not None:
                for _doc in sub_docs:
                    _doc.metadata = {
                        k: _doc.metadata[k] for k in self.child_metadata_fields
                    }
            for _doc in sub_docs:
                _doc.metadata[self.id_key] = _id
            docs.extend(sub_docs)
            full_docs.append((_id, doc))
        self.vectorstore.add_documents(docs)
        if add_to_docstore:
            self.docstore.mset(full_docs)

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        preprocessed_query = pre_embedding_process(query)
        print(f"Processed query: {preprocessed_query}")
        return super(ParentDocumentPreprocessRetriever, self)._get_relevant_documents(preprocessed_query, run_manager=run_manager)

    def _aget_relevant_documents(self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun) -> Coroutine[Any, Any, List[Document]]:
        preprocessed_query = pre_embedding_process(query)
        print(f"Processed query: {preprocessed_query}")
        return super()._aget_relevant_documents(preprocessed_query, run_manager=run_manager)


In [108]:
from langchain.storage import InMemoryStore
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import ParentDocumentRetriever
from faiss import IndexFlatL2


query = QUERIES[0]

parent_splitter = RecursiveCharacterTextSplitter.from_language(
    Language.MARKDOWN,
    chunk_size=2000,
    chunk_overlap=400,
    length_function=len,
)

child_splitter = RecursiveCharacterTextSplitter.from_language(
    Language.MARKDOWN,
    chunk_size=400,
    chunk_overlap=100,
    length_function=len,
)
embedding = get_embeddings()
# store = FAISS(embedding, IndexFlatL2(0), InMemoryDocstore(), {})
store = FAISS.from_documents([raw_documents[0]], core_embeddings_model)
retriever = ParentDocumentPreprocessRetriever(
    vectorstore=store,
    docstore=InMemoryStore(),
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(raw_documents, ids=None)
retrieved_docs = retriever.invoke(query)
print(retrieved_docs[0].page_content)


embedding_vector = core_embeddings_model.embed_query(query)
docs = store.similarity_search_with_score_by_vector(embedding_vector, k = 4)

print("From store:")
for page in docs:
  print(page[0].dict())

Processed query: deal with memory leak in prod
# 2016–09–30 Other (~resolved🎉?) Fires

Memory leak in prod

Update / Tentatively Resolved
Appears that the cause was not in the suspect list below (but in the deploy) https://github.com/NoRedInk/NoRedInk/pull/13550 which is now the primary suspect with a fix deployed in https://github.com/NoRedInk/NoRedInk/pull/13703 that thus far appears to have removed network correlated memory spikes.

Things we can do

Deploy/rollback to production-2016-09-29-0434

Revert the suspect PRs

Suspect PRs
From @Joshua L (sorted by suspicion)

[ ] topic code cache rejiggering 
    - https://github.com/NoRedInk/NoRedInk/pull/13564
        - Noah doesn’t know enough to review (but tried his best anyway)
        - Removes code plan from practices
        - makes a kernel for generating a new random seed from an old one
        - Makes code plan a module
        - caches a topic id with a user id + code index on every call to next topic code with Resque.redis
 

In [126]:
from langchain.retrievers import RePhraseQueryRetriever

retriever_from_llm = RePhraseQueryRetriever.from_llm(
    retriever=store.as_retriever(), llm=llm
)

docs = retriever_from_llm.invoke(
    "Hi I'm Lance. What are the responsibilities of the First Captain during a fire?"
)

print(docs)

[Document(page_content='grammar_quiz_questions table retrieved with index distribution_idx mysql extra pas retrieve row in sorted order which cause poor performance sometimes unavoidable speed up query querying only field within index create index includes every field in query including primary key', metadata={'source': '.content/Engineering/Fires/Old_Fires/Oct_11_2018,_100_CPU_Fire.md', 'doc_id': '3e447047-1af2-4850-b475-ea187b56e734'}), Document(page_content='index_topic_usages_on_user_id_and_topic_id speed up query querying only field within index create index includes every field in query including primary key approximately row table were scanned mysql lessonimage find m query time cpm throughput trace from teach assignablescontroller new', metadata={'source': '.content/Engineering/Fires/Old_Fires/Oct_11_2018,_100_CPU_Fire.md', 'doc_id': 'e107665a-5712-4192-881f-38587f7037b9'}), Document(page_content='mysql extra pas retrieve row in sorted order which cause poor performance sometim

In [117]:
from flashrank import Ranker, RerankRequest
from random import shuffle
# Nano (~4MB), blazing fast model & competitive performance (ranking precision).
ranker = Ranker()

query = "What should I do if there are problems with the haskell quiz engine?"
passages = [
    {
        "id": 0,
        "text": "If the database is at 100% CPU\nYou can try slowing down the quiz engine as it’s usually the biggest source of DB time consumption\nYou can try spinning down a few Monolith servers to cause request queuing and lower throughput to the database\n\nIf not, and request queueing is already very high, you can try spinning up more Monolith servers to absorb request queueing\n\nHaskell Quiz Engine Problems?\n\nHow to tell?\n\nweb external is super high\n\n“External Services” tab shows that HQE has the “slowest average response time” (it usually is the most time consuming because of volume of traffic).\n\nWhat to do?\n\ndisable all HQE feature flags (off is safer)\n\ncheck in on HQE firefighting notes\n\nFighting a DDoS attack\n\nHow to tell?\n\nAn unusual bump in traffic (see the Throughput  chart in the Summary page in NewRelic)",
        "meta": {"source": ".content/Engineering/Fires/Firefighting_Resources.md"},
    },
    {
        "id": 1,
        "text": "Throughput\nDo we have a huge spike in throughput? 200k RPM and above might mean we’re under a DDoS attack\n➡️ Check out what to do here: +Firefighting Resources: Fighting-a-DDoS-attack\n\nError rate\nAre error spiking? Like 1% or more\n➡️ Go to Errors tab\n➡️ Check out Bugsnag (finer grained timeline)\n\nExternal Services\n\nServices by total response time\nAny service started taking a longer time than usual?\nA Haskell service?\n➡️ Go to the specific service’s dashboard in NewRelic and do a similar analysis there (what made it slow? etc)\n➡️ Also, open the Haskell Services in Kubernete dashboard in Datadog, because Kubernetes might be at fault\nWhat services are in Kubernetes?\nDrafts\nQuiz Engine\nTutorials",
        "meta": {"source": ".content/Engineering/Fires/Fire_triage_drill.md"},
    },
    {
        "id": 2,
        "text": "Databases\n\nCheck if any DB operations are spiking\nYou can click ActiveRecord or MySQL on the top left to see more activerecord/mysql queries in the graph, if nothing is spiking\nSometimes everything is spiking, or lots of things are, and in these cases it’s more likely a general MySQL problem\n➡️ Diagnose further on RDS Performance Insights\n\nClick on the spiky operation\n\nDatadog\n\nDatadog\n\nTwo main things here:\n\nHaskell in Kubernetes Dashboard\nA dashboard with 90% of what you need to triage fires in Haskell services deployed to Kubernetes\n\nProcess Monitor\nLike htop, but in Datadog. Useful for seeing Threads per process and what % a process is taking over the entire machine.\n\nHaskell in Kubernetes Dashboard\n\n🔗 Link\n\nIs CPU usage in the monolith too high? Above 48% is too high.\n\nIs load average for Kubernetes too high?\nℹ️ It might indicate contention in one or more services\nℹ️ If it correlates with IOWait spikes, it might indicate network/disk contention",
        "meta": {"source": ".content/Engineering/Fires/Fire_triage_drill.md"},
    },
    {
        "id": 3,
        "text": 'Takeaways\n\nRSS feed for Canvas API changes could directly create new Linear issues\nThere was an update about this API change: https://community.canvaslms.com/t5/Canvas-Change-Log/Canvas-Platform-Breaking-Changes/ta-p/262015 — "These numerical variable substitutions will be deprecated and changed to string values on 2022-10-15"\nDid someone at NRI receive an email alert about this from Canvas? Where did it get sent to and why didn’t anyone see it?\nSlack integration is /feed to add an RSS feed to a slack channel\n\nHaskell deploy quite slow, ideally should be under 20 minutes\nAnd very common to need to retry Haskell deploy at least one more time before it works\n\nShould be able to deploy to any Haskell services without it hanging on tests if during a fire!',
        "meta": {
            "source": ".content/Engineering/Fires/Old_Fires/2022-10-17_Canvas_API_breaks_assignments.md"
        },
    },
    {
        "id": 4,
        "text": "3:10 Uh oh! We deploy Haskell services before we start the demo deploy. During that period, \n(a) services and the monolith are out of sync \n(b) the demo account pool is wiped\nSuggestion from Tessa: we might turn on the drafts service killswitch and the hypothetical HQE killswitch before the deploy, so that the bits of the site that we expect to have broken are explicitly down.\n\nKristine: there’s an agreement with CS to not use demo during this period, so it’s not necessary.\n\nTessa, editorializing: Kind of interesting to have a 2-hour extreme version of our usual production deploy problems! I wonder if Foxen are aware of the situation 🤔",
        "meta": {
            "source": ".content/Engineering/Fires/Old_Fires/Fire_Dec_8,_2020_-_Demo_deploy_stuck.md"
        },
    },
    {
        "id": 5,
        "text": "[ ] Currently deployed revision When was a PR deployed? Turn off the service Change configuration / environment variable Deploy a hotfix Safely reboot an instance (without causing user pain / data loss / etc) Monolith Quick: Tip of the [production](https://github.com/NoRedInk/NoRedInk/commits/production) branch Most accurate: /opt/noredink/current/REVISION of any app server Search for the PR number or branch name in #eng-notify-info For demo, search in #demo-status - maintenance mode - throttle requests to the quiz engine - flipper has various feature flags named *killswitch* that turns off a particular feature Follow the wiki page for adding an env var. In a pinch, update the SSM param with awscli and propagate the change following the subsequent steps. Follow the regular Haskell Monorepo deploy process +Debugging live on Kubernetes: Restarting-a-pod +Debugging live on Kubernetes: Safely-reboot-an-instance Reports edeliver or ssh to completely stop the service Follow this doc Deploy",
        "meta": {"source": ".content/Engineering/Fires/Firefighting_Resources.md"},
    },
]
# passages = [
#     {
#         "text": "If the database is at 100% CPU\nYou can try slowing down the quiz engine as it’s usually the biggest source of DB time consumption\nYou can try spinning down a few Monolith servers to cause request queuing and lower throughput to the database\n\nIf not, and request queueing is already very high, you can try spinning up more Monolith servers to absorb request queueing\n\nHaskell Quiz Engine Problems?\n\nHow to tell?\n\nweb external is super high\n\n“External Services” tab shows that HQE has the “slowest average response time” (it usually is the most time consuming because of volume of traffic).\n\nWhat to do?\n\ndisable all HQE feature flags (off is safer)\n\ncheck in on HQE firefighting notes\n\nFighting a DDoS attack\n\nHow to tell?\n\nAn unusual bump in traffic (see the Throughput  chart in the Summary page in NewRelic)",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Firefighting_Resources.md"
#         },
#         "id": "g"
#     },
#     {
#         "text": "Throughput\nDo we have a huge spike in throughput? 200k RPM and above might mean we’re under a DDoS attack\n➡️ Check out what to do here: +Firefighting Resources: Fighting-a-DDoS-attack\n\nError rate\nAre error spiking? Like 1% or more\n➡️ Go to Errors tab\n➡️ Check out Bugsnag (finer grained timeline)\n\nExternal Services\n\nServices by total response time\nAny service started taking a longer time than usual?\nA Haskell service?\n➡️ Go to the specific service’s dashboard in NewRelic and do a similar analysis there (what made it slow? etc)\n➡️ Also, open the Haskell Services in Kubernete dashboard in Datadog, because Kubernetes might be at fault\nWhat services are in Kubernetes?\nDrafts\nQuiz Engine\nTutorials",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Fire_triage_drill.md"
#         },
#         "id": "d"
#     },
#     {
#         "text": "Databases\n\nCheck if any DB operations are spiking\nYou can click ActiveRecord or MySQL on the top left to see more activerecord/mysql queries in the graph, if nothing is spiking\nSometimes everything is spiking, or lots of things are, and in these cases it’s more likely a general MySQL problem\n➡️ Diagnose further on RDS Performance Insights\n\nClick on the spiky operation\n\nDatadog\n\nDatadog\n\nTwo main things here:\n\nHaskell in Kubernetes Dashboard\nA dashboard with 90% of what you need to triage fires in Haskell services deployed to Kubernetes\n\nProcess Monitor\nLike htop, but in Datadog. Useful for seeing Threads per process and what % a process is taking over the entire machine.\n\nHaskell in Kubernetes Dashboard\n\n🔗 Link\n\nIs CPU usage in the monolith too high? Above 48% is too high.\n\nIs load average for Kubernetes too high?\nℹ️ It might indicate contention in one or more services\nℹ️ If it correlates with IOWait spikes, it might indicate network/disk contention",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Fire_triage_drill.md"
#         },
#         "id": "f"
#     },
#     {
#         "text": "Takeaways\n\nRSS feed for Canvas API changes could directly create new Linear issues\nThere was an update about this API change: https://community.canvaslms.com/t5/Canvas-Change-Log/Canvas-Platform-Breaking-Changes/ta-p/262015 — \"These numerical variable substitutions will be deprecated and changed to string values on 2022-10-15\"\nDid someone at NRI receive an email alert about this from Canvas? Where did it get sent to and why didn’t anyone see it?\nSlack integration is /feed to add an RSS feed to a slack channel\n\nHaskell deploy quite slow, ideally should be under 20 minutes\nAnd very common to need to retry Haskell deploy at least one more time before it works\n\nShould be able to deploy to any Haskell services without it hanging on tests if during a fire!",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Old_Fires/2022-10-17_Canvas_API_breaks_assignments.md"
#         },
#         "id": "j"
#     },
#     {
#         "text": "3:10 Uh oh! We deploy Haskell services before we start the demo deploy. During that period, \n(a) services and the monolith are out of sync \n(b) the demo account pool is wiped\nSuggestion from Tessa: we might turn on the drafts service killswitch and the hypothetical HQE killswitch before the deploy, so that the bits of the site that we expect to have broken are explicitly down.\n\nKristine: there’s an agreement with CS to not use demo during this period, so it’s not necessary.\n\nTessa, editorializing: Kind of interesting to have a 2-hour extreme version of our usual production deploy problems! I wonder if Foxen are aware of the situation 🤔",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Old_Fires/Fire_Dec_8,_2020_-_Demo_deploy_stuck.md"
#         },
#         "id": "l"
#     },
#     {
#         "text": "[ ] Currently deployed revision When was a PR deployed? Turn off the service Change configuration / environment variable Deploy a hotfix Safely reboot an instance (without causing user pain / data loss / etc) Monolith Quick: Tip of the [production](https://github.com/NoRedInk/NoRedInk/commits/production) branch Most accurate: /opt/noredink/current/REVISION of any app server Search for the PR number or branch name in #eng-notify-info For demo, search in #demo-status - maintenance mode - throttle requests to the quiz engine - flipper has various feature flags named *killswitch* that turns off a particular feature Follow the wiki page for adding an env var. In a pinch, update the SSM param with awscli and propagate the change following the subsequent steps. Follow the regular Haskell Monorepo deploy process +Debugging live on Kubernetes: Restarting-a-pod +Debugging live on Kubernetes: Safely-reboot-an-instance Reports edeliver or ssh to completely stop the service Follow this doc Deploy",
#         "metadata": {
#             "source": ".content/Engineering/Fires/Firefighting_Resources.md"
#         },
#         "id": "v"
#     }
# ]
shuffle(passages)
rerankrequest = RerankRequest(query=query, passages=passages)
results = ranker.rerank(rerankrequest)

import pprint
pprint.pp(results)

Running pairwise ranking..
[{'id': 0,
  'text': 'If the database is at 100% CPU\n'
          'You can try slowing down the quiz engine as it’s usually the '
          'biggest source of DB time consumption\n'
          'You can try spinning down a few Monolith servers to cause request '
          'queuing and lower throughput to the database\n'
          '\n'
          'If not, and request queueing is already very high, you can try '
          'spinning up more Monolith servers to absorb request queueing\n'
          '\n'
          'Haskell Quiz Engine Problems?\n'
          '\n'
          'How to tell?\n'
          '\n'
          'web external is super high\n'
          '\n'
          '“External Services” tab shows that HQE has the “slowest average '
          'response time” (it usually is the most time consuming because of '
          'volume of traffic).\n'
          '\n'
          'What to do?\n'
          '\n'
          'disable all HQE feature flags (off is safer)\n'
          '

In [125]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)
chain.invoke({"input_documents": raw_documents[:2]})



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"🔥 Fire triage drill

Note: This document is not intended to be consumed by itself. I’m personally introducing fire fighters to this drill at the start of their on-call cycle, and the document is here for a) reference for me b) reference for fire fighters after our meeting.

The fire drill starts at the NewRelic APM main screen for the Monolith. Go through the numbers on the image, reading their description below, and follow the arrows ➡️ .

NewRelic

Monolith NewRelic main screen

🔗 Link

Request breakdown
Did something start taking up a larger share of request time?
External services
➡️ Go to the External Services tab in the right and figure out which service


DB
➡️ Go to the Databases tab and see whether some transactions started spiking
➡️ Open RDS Performance Insights for higher resolution data


Queueing or Ruby Slowness
➡️ Open L

{'input_documents': [Document(page_content='🔥 Fire triage drill\n\nNote: This document is not intended to be consumed by itself. I’m personally introducing fire fighters to this drill at the start of their on-call cycle, and the document is here for a) reference for me b) reference for fire fighters after our meeting.\n\nThe fire drill starts at the NewRelic APM main screen for the Monolith. Go through the numbers on the image, reading their description below, and follow the arrows ➡️ .\n\nNewRelic\n\nMonolith NewRelic main screen\n\n🔗 Link\n\nRequest breakdown\nDid something start taking up a larger share of request time?\nExternal services\n➡️ Go to the External Services tab in the right and figure out which service\n\n\nDB\n➡️ Go to the Databases tab and see whether some transactions started spiking\n➡️ Open RDS Performance Insights for higher resolution data\n\n\nQueueing or Ruby Slowness\n➡️ Open Load Balancers Dashboard\n➡️ Opsworks time-based instances and load-based instances\n

In [123]:
s = 'Here is a refined summary based on the provided context:\n\n**Firefighter\'s Checklist**\n\nAs a firefighter, your role is to help resolve fires (technical issues) in our system. Here are some key steps to follow:\n\n1. **Get Current Task**: Ensure you have a current task assigned by the captain.\n2. **Make Product Decisions**: If needed, pull in someone from #ask-support or #ask-product for guidance.\n3. **Update Channels**: Notify all relevant channels (including #fires) of any updates or news.\n4. **Keep Notes Up-to-Date**: Update notes in Engineering > Fires > In Progress regularly.\n5. **Escalate if Stuck**: If you\'re stuck, escalate to Director of Engineering and/or Head of Product by phone for guidance.\n\n**When Fire is Resolved**\n\n1. **Update Channels**: Notify all relevant channels (including #fires) of the fire\'s resolution.\n2. **Captain Hand-Off**: Pin the new captain to #fix-the-fire and ensure notes are up-to-date and convey relevant information.\n\n**Last Captain**\n\n1. **Move Notes**: Move notes from Engineering > Fires > In Progress to Engineering > Fires > Old Fires.\n2. **Write Fire Follow-up**: Write a follow-up story (bug type) with the label "fire follow-up" and link to the write-up template.\n3. **Assign Eligible Writers**: Assign all firefighters as eligible people who can write the follow-up report.\n\n**How Can I Help as a Non-ops?**\n\n1. **Pair/Observation**: Offer to pair or observe with ops to transfer knowledge.\n2. **Take Notes**: Offer to take notes and ask questions to understand the process better.\n\nThis refined summary provides a clear checklist for firefighters to follow, ensuring that fires are resolved efficiently and effectively.'
pprint.pp(s)

('Here is a refined summary based on the provided context:\n'
 '\n'
 "**Firefighter's Checklist**\n"
 '\n'
 'As a firefighter, your role is to help resolve fires (technical issues) in '
 'our system. Here are some key steps to follow:\n'
 '\n'
 '1. **Get Current Task**: Ensure you have a current task assigned by the '
 'captain.\n'
 '2. **Make Product Decisions**: If needed, pull in someone from #ask-support '
 'or #ask-product for guidance.\n'
 '3. **Update Channels**: Notify all relevant channels (including #fires) of '
 'any updates or news.\n'
 '4. **Keep Notes Up-to-Date**: Update notes in Engineering > Fires > In '
 'Progress regularly.\n'
 "5. **Escalate if Stuck**: If you're stuck, escalate to Director of "
 'Engineering and/or Head of Product by phone for guidance.\n'
 '\n'
 '**When Fire is Resolved**\n'
 '\n'
 '1. **Update Channels**: Notify all relevant channels (including #fires) of '
 "the fire's resolution.\n"
 '2. **Captain Hand-Off**: Pin the new captain to #fix-the-fir